# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Step 1 DataSet Selection

In [ ]:
file = "state_GA.csv" 
data = pd.read_csv(file)

data_reduced = data[['lei', 'derived_ethnicity', 'derived_race', 'derived_sex', 'loan_amount', 'debt_to_income_ratio', 'loan_purpose', 'loan_term', 'action_taken', 'denial_reason-1', 'denial_reason-2', 
                     'income', 'applicant_age', 'total_loan_costs', 'interest_rate', 'applicant_race-1', 'applicant_race-2', 'applicant_sex','co-applicant_sex']]

In [ ]:
## Only Need to run once
data_reduced.to_csv("state_GA_reduced.csv", index=False)
hmda_data = pd.read_csv("state_GA_reduced.csv")

In [ ]:
sex_map = {
    1: "Male",
    2: "Female",
    6: "Applicant selected both male and female"
}

race_map = {
    1: "American Indian or Alaska Native",
    2: "Asian",
    21: "Asian Indian",
    22: "Chinese",
    23: "Filipino",
    24: "Japanese",
    25: "Korean",
    26: "Vietnamese",
    27: "Other Asian",
    3: "Black or African American",
    4: "Native Hawaiian or Other Pacific Islander",
    41: "Native Hawaiian",
    42: "Guamanian or Chamorro",
    43: "Samoan",
    44: "Other Pacific Islander",
    5: "White"
}

data_reduced['applicant_race-1'] = pd.to_numeric(data_reduced['applicant_race-1'], errors='coerce')
data_reduced['applicant_race-2'] = pd.to_numeric(data_reduced['applicant_race-2'], errors='coerce')
data_reduced['applicant_sex'] = pd.to_numeric(data_reduced['applicant_sex'], errors='coerce')
data_reduced['co-applicant_sex'] = pd.to_numeric(data_reduced['co-applicant_sex'], errors='coerce')

filtered = data_reduced[
    data_reduced['applicant_race-1'].isin(race_map.keys()) &
    (
        data_reduced['applicant_race-2'].isna() |
        data_reduced['applicant_race-2'].isin(race_map.keys())
    ) &
    data_reduced['applicant_sex'].isin(sex_map.keys()) &
    (
        data_reduced['co-applicant_sex'].isna() |
        data_reduced['co-applicant_sex'].isin(sex_map.keys())
    )

]

filtered['race_1_str'] = filtered['applicant_race-1'].map(race_map)
filtered['race_2_str'] = filtered['applicant_race-2'].map(race_map)
def combine_races_str(row):
    if pd.isna(row['race_2_str']) or row['race_2_str'] == "":
        return row['race_1_str']
    return f"{row['race_1_str']}, {row['race_2_str']}"
def combine_sexs_str(row):
    if pd.isna(row['co-applicant_sex']) or row['co-applicant_sex'] == "":
        return row['applicant_sex']
    return f"{row['applicant_sex']}, {row['co-applicant_sex']}"

filtered['derived_race_new'] = filtered.apply(combine_races_str, axis=1)

filtered['applicant_sex'] = filtered['applicant_sex'].map(sex_map)
filtered['co-applicant_sex'] = filtered['co-applicant_sex'].map(sex_map)

filtered['derived_sex_new'] = filtered.apply(combine_sexs_str, axis=1)

filtered = filtered[filtered['action_taken'] != 6]
filtered = filtered[filtered['interest_rate'] != 'Exempt']


filtered['favorable_action_taken'] = filtered['action_taken'].apply(
    lambda x: 1 if x in [1, 2, 8] else (0 if x in [3, 4, 5, 7] else pd.NA)
)
filtered['interest_rate'].unique()
filtered['favorable_interest_rate'] = filtered['interest_rate'].apply(
    lambda x: 1 if float(x) <= 7.5 else 0)

unique_races = sorted(filtered['derived_race_new'].unique())
unique_sexes = sorted(filtered['derived_sex_new'].unique())

# New encoding dictionaries
final_race_encoding = {race: i for i, race in enumerate(unique_races)}
final_sex_encoding = {sex: i for i, sex in enumerate(unique_sexes)}

filtered['derived_race_encoded'] = filtered['derived_race_new'].map(final_race_encoding)
filtered['derived_sex_encoded'] = filtered['derived_sex_new'].map(final_sex_encoding)


filtered.to_csv("state_GA_reduced_encoded.csv", index=False)

# Analysis for Steps 3 and 4

## Load Data

In [2]:
try:
    df = pd.read_csv('state_GA_reduced_encoded.csv')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: 'state_GA_reduced_encoded.csv' not found.")

Dataset loaded successfully.


## Data Summary

In [3]:
print("\n--- Initial Data Summary ---")
print(f"Original dataset shape: {df.shape}")
print("\nData Info:")
df.info()
print("\nValue Counts for Action Taken:")
print(df['action_taken'].value_counts())


--- Initial Data Summary ---
Original dataset shape: (109250, 26)

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109250 entries, 0 to 109249
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   lei                   109250 non-null  object 
 1   derived_ethnicity     109250 non-null  object 
 2   derived_race          109250 non-null  object 
 3   derived_sex           109250 non-null  object 
 4   loan_amount           109250 non-null  float64
 5   debt_to_income_ratio  84112 non-null   object 
 6   loan_purpose          109250 non-null  int64  
 7   loan_term             108220 non-null  object 
 8   action_taken          109250 non-null  int64  
 9   denial_reason-1       109250 non-null  int64  
 10  denial_reason-2       3930 non-null    float64
 11  income                104831 non-null  float64
 12  applicant_age         109250 non-null  object 
 13  total_loan_costs      546

## Data Cleaning

In [4]:
df['interest_rate'] = pd.to_numeric(df['interest_rate'], errors='coerce')
df['favorable_interest_rate'] = np.where(df['interest_rate'] < 7.5, 1, 0)
print("\nDependent variables prepared.")

FEATURES = ['loan_amount', 'income', 'derived_race_encoded', 'derived_sex_encoded']

df['income'] = pd.to_numeric(df['income'], errors='coerce')
original_rows = len(df)
df.dropna(subset=FEATURES, inplace=True)
print(f"\nDropped {original_rows - len(df)} rows with missing values in feature columns to prepare for modeling.")
print(f"Shape of data ready for modeling: {df.shape}")


sex_df = df[df['applicant_sex'].isin(['Male', 'Female'])].copy()
race_df = df[df['derived_race_new'].isin(['White', 'Black or African American'])].copy()
privileged_sex_group = {'applicant_sex': 'Male'}
unprivileged_sex_group = {'applicant_sex': 'Female'}
privileged_race_group = {'derived_race_new': 'White'}
unprivileged_race_group = {'derived_race_new': 'Black or African American'}
print("\n--- Protected Class Subgroup Counts ---")
print("\nSex Analysis Subgroups (sex_df):")
print(sex_df['applicant_sex'].value_counts())
print("\nRace Analysis Subgroups (race_df):")
print(race_df['derived_race_new'].value_counts())


Dependent variables prepared.

Dropped 4419 rows with missing values in feature columns to prepare for modeling.
Shape of data ready for modeling: (104831, 27)

--- Protected Class Subgroup Counts ---

Sex Analysis Subgroups (sex_df):
Male      68120
Female    36442
Name: applicant_sex, dtype: int64

Race Analysis Subgroups (race_df):
White                        75422
Black or African American    19982
Name: derived_race_new, dtype: int64


# Step 3

### Manual Fairness Metric Calculation

In [ ]:
# Manual computation of fairness metrics
def compute_manual_fairness_metrics(df, protected_attribute, dependent_variable, privileged_group, unprivileged_group, weights_col=None):
    priv_key, priv_val = list(privileged_group.items())[0]
    unpriv_key, unpriv_val = list(unprivileged_group.items())[0]
    df_priv = df[df[priv_key] == priv_val]
    df_unpriv = df[df[unpriv_key] == unpriv_val]
    
    if weights_col and not df_priv.empty and not df_unpriv.empty:
        rate_priv = (df_priv[dependent_variable] * df_priv[weights_col]).sum() / df_priv[weights_col].sum()
        rate_unpriv = (df_unpriv[dependent_variable] * df_unpriv[weights_col]).sum() / df_unpriv[weights_col].sum()
    elif not df_priv.empty and not df_unpriv.empty:
        rate_priv = df_priv[dependent_variable].mean()
        rate_unpriv = df_unpriv[dependent_variable].mean()
    else:
        return {'Statistical Parity Difference': np.nan, 'Disparate Impact': np.nan}
        
    spd = rate_unpriv - rate_priv
    di = rate_unpriv / (rate_priv + 1e-7)
    return {'Statistical Parity Difference': spd, 'Disparate Impact': di}

### Apply Reweighting

In [ ]:
# Manual application of reweighting
def apply_reweighting(df, protected_attribute, dependent_variable, privileged_group, unprivileged_group):
    df_new = df.copy()
    priv_key, priv_val = list(privileged_group.items())[0]
    unpriv_key, unpriv_val = list(unprivileged_group.items())[0]
    priv_fav = (df_new[priv_key] == priv_val) & (df_new[dependent_variable] == 1)
    priv_unfav = (df_new[priv_key] == priv_val) & (df_new[dependent_variable] == 0)
    unpriv_fav = (df_new[unpriv_key] == unpriv_val) & (df_new[dependent_variable] == 1)
    unpriv_unfav = (df_new[unpriv_key] == unpriv_val) & (df_new[dependent_variable] == 0)
    N = len(df_new)
    p_priv = (df_new[priv_key] == priv_val).sum() / N
    p_unpriv = (df_new[unpriv_key] == unpriv_val).sum() / N
    p_fav = (df_new[dependent_variable] == 1).sum() / N
    p_unfav = (df_new[dependent_variable] == 0).sum() / N
    p_priv_fav = priv_fav.sum() / N; p_priv_unfav = priv_unfav.sum() / N
    p_unpriv_fav = unpriv_fav.sum() / N; p_unpriv_unfav = unpriv_unfav.sum() / N
    w_priv_fav = (p_priv * p_fav) / p_priv_fav if p_priv_fav > 0 else 1.0
    w_priv_unfav = (p_priv * p_unfav) / p_priv_unfav if p_priv_unfav > 0 else 1.0
    w_unpriv_fav = (p_unpriv * p_fav) / p_unpriv_fav if p_unpriv_fav > 0 else 1.0
    w_unpriv_unfav = (p_unpriv * p_unfav) / p_unpriv_unfav if p_unpriv_unfav > 0 else 1.0
    df_new['sample_weight'] = 1.0
    df_new.loc[priv_fav, 'sample_weight'] = w_priv_fav
    df_new.loc[priv_unfav, 'sample_weight'] = w_priv_unfav
    df_new.loc[unpriv_fav, 'sample_weight'] = w_unpriv_fav
    df_new.loc[unpriv_unfav, 'sample_weight'] = w_unpriv_unfav
    return df_new

## Preprocess and Mitigate Data

In [7]:
original_metrics = {
    'Sex vs. Action Taken': compute_manual_fairness_metrics(sex_df, 'applicant_sex', 'action_taken', privileged_sex_group, unprivileged_sex_group),
    'Race vs. Action Taken': compute_manual_fairness_metrics(race_df, 'derived_race_new', 'action_taken', privileged_race_group, unprivileged_race_group),
    'Sex vs. Favorable Interest Rate': compute_manual_fairness_metrics(sex_df, 'applicant_sex', 'favorable_interest_rate', privileged_sex_group, unprivileged_sex_group),
    'Race vs. Favorable Interest Rate': compute_manual_fairness_metrics(race_df, 'derived_race_new', 'favorable_interest_rate', privileged_race_group, unprivileged_race_group)
}
print(pd.DataFrame.from_dict(original_metrics, orient='index'))

                                  Statistical Parity Difference  \
Sex vs. Action Taken                                   0.082922   
Race vs. Action Taken                                  0.367557   
Sex vs. Favorable Interest Rate                       -0.004527   
Race vs. Favorable Interest Rate                      -0.073302   

                                  Disparate Impact  
Sex vs. Action Taken                      1.042136  
Race vs. Action Taken                     1.191670  
Sex vs. Favorable Interest Rate           0.989941  
Race vs. Favorable Interest Rate          0.839495  


### Reweight

In [8]:
sex_df_transformed = apply_reweighting(sex_df, 'applicant_sex', 'action_taken', privileged_sex_group, unprivileged_sex_group)
race_df_transformed = apply_reweighting(race_df, 'derived_race_new', 'action_taken', privileged_race_group, unprivileged_race_group)
print("\nReweighting applied to create transformed datasets.")




Reweighting applied to create transformed datasets.


### Transformed Data Metrics

In [9]:
transformed_metrics = {
    'Sex vs. Action Taken': compute_manual_fairness_metrics(sex_df_transformed, 'applicant_sex', 'action_taken', privileged_sex_group, unprivileged_sex_group, weights_col='sample_weight'),
    'Race vs. Action Taken': compute_manual_fairness_metrics(race_df_transformed, 'derived_race_new', 'action_taken', privileged_race_group, unprivileged_race_group, weights_col='sample_weight'),
    'Sex vs. Favorable Interest Rate': compute_manual_fairness_metrics(sex_df_transformed, 'applicant_sex', 'favorable_interest_rate', privileged_sex_group, unprivileged_sex_group, weights_col='sample_weight'),
    'Race vs. Favorable Interest Rate': compute_manual_fairness_metrics(race_df_transformed, 'derived_race_new', 'favorable_interest_rate', privileged_race_group, unprivileged_race_group, weights_col='sample_weight')
}
print(pd.DataFrame.from_dict(transformed_metrics, orient='index'))


                                  Statistical Parity Difference  \
Sex vs. Action Taken                                   0.048991   
Race vs. Action Taken                                  0.201853   
Sex vs. Favorable Interest Rate                        0.004373   
Race vs. Favorable Interest Rate                      -0.028255   

                                  Disparate Impact  
Sex vs. Action Taken                      1.024750  
Race vs. Action Taken                     1.103655  
Sex vs. Favorable Interest Rate           1.009780  
Race vs. Favorable Interest Rate          0.937112  


# Step 4 Mitigating Bias

### Data Splitting

#### Sex Analysis

In [10]:
X_sex = sex_df[FEATURES]
y_sex = sex_df['action_taken']
X_sex_train, X_sex_test, y_sex_train, y_sex_test = train_test_split(X_sex, y_sex, test_size=0.2, random_state=42, stratify=y_sex)
X_sex_transformed = sex_df_transformed[FEATURES]
y_sex_transformed = sex_df_transformed['action_taken']
weights_sex_transformed = sex_df_transformed['sample_weight']
X_sex_train_t, X_sex_test_t, y_sex_train_t, y_sex_test_t, w_sex_train_t, w_sex_test_t = train_test_split(
X_sex_transformed, y_sex_transformed, weights_sex_transformed, test_size=0.2, random_state=42, stratify=y_sex_transformed)

#### Race Analysis

In [11]:

X_race = race_df[FEATURES]
y_race = race_df['action_taken']
X_race_train, X_race_test, y_race_train, y_race_test = train_test_split(X_race, y_race, test_size=0.2, random_state=42, stratify=y_race)
X_race_transformed = race_df_transformed[FEATURES]
y_race_transformed = race_df_transformed['action_taken']
weights_race_transformed = race_df_transformed['sample_weight']
X_race_train_t, X_race_test_t, y_race_train_t, y_race_test_t, w_race_train_t, w_race_test_t = train_test_split(
X_race_transformed, y_race_transformed, weights_race_transformed, test_size=0.2, random_state=42, stratify=y_race_transformed)

## Model Training

#### Original Data Model

In [ ]:
scaler = StandardScaler()

model_sex_original = LogisticRegression(random_state=42, class_weight='balanced')
model_sex_original.fit(scaler.fit_transform(X_sex_train), y_sex_train)
sex_test_preds_original = model_sex_original.predict(scaler.transform(X_sex_test))
print(f"Accuracy of original model (Sex): {accuracy_score(y_sex_test, sex_test_preds_original):.2f}")

model_race_original = LogisticRegression(random_state=42, class_weight='balanced')
model_race_original.fit(scaler.fit_transform(X_race_train), y_race_train)
race_test_preds_original = model_race_original.predict(scaler.transform(X_race_test))
print(f"Accuracy of original model (Race): {accuracy_score(y_race_test, race_test_preds_original):.2f}")

Accuracy of original model (Sex): 0.23
Accuracy of original model (Race): 0.23
Accuracy of transformed model (Sex): 0.62
Accuracy of transformed model (Race): 0.62


#### Transformed Data Models

In [ ]:
model_sex_transformed = LogisticRegression(random_state=42)
model_sex_transformed.fit(scaler.fit_transform(X_sex_train_t), y_sex_train_t, sample_weight=w_sex_train_t)
sex_test_preds_transformed = model_sex_transformed.predict(scaler.transform(X_sex_test_t))
print(f"Accuracy of transformed model (Sex): {accuracy_score(y_sex_test_t, sex_test_preds_transformed):.2f}")


model_race_transformed = LogisticRegression(random_state=42)
model_race_transformed.fit(scaler.fit_transform(X_race_train_t), y_race_train_t, sample_weight=w_race_train_t)
race_test_preds_transformed = model_race_transformed.predict(scaler.transform(X_race_test_t))
print(f"Accuracy of transformed model (Race): {accuracy_score(y_race_test_t, race_test_preds_transformed):.2f}")

## Fairness Metrics

In [13]:
sex_test_df_original = sex_df.loc[X_sex_test.index].copy()
sex_test_df_original['prediction'] = sex_test_preds_original
race_test_df_original = race_df.loc[X_race_test.index].copy()
race_test_df_original['prediction'] = race_test_preds_original
sex_test_df_transformed = sex_df_transformed.loc[X_sex_test_t.index].copy()
sex_test_df_transformed['prediction'] = sex_test_preds_transformed
race_test_df_transformed = race_df_transformed.loc[X_race_test_t.index].copy()
race_test_df_transformed['prediction'] = race_test_preds_transformed

original_pred_metrics = {
    'Sex vs. Prediction': compute_manual_fairness_metrics(sex_test_df_original, 'applicant_sex', 'prediction', privileged_sex_group, unprivileged_sex_group),
    'Race vs. Prediction': compute_manual_fairness_metrics(race_test_df_original, 'derived_race_new', 'prediction', privileged_race_group, unprivileged_race_group)
}
print(pd.DataFrame.from_dict(original_pred_metrics, orient='index'))


transformed_pred_metrics = {
    'Sex vs. Prediction': compute_manual_fairness_metrics(sex_test_df_transformed, 'applicant_sex', 'prediction', privileged_sex_group, unprivileged_sex_group),
    'Race vs. Prediction': compute_manual_fairness_metrics(race_test_df_transformed, 'derived_race_new', 'prediction', privileged_race_group, unprivileged_race_group)
}

print(pd.DataFrame.from_dict(transformed_pred_metrics, orient='index'))

                     Statistical Parity Difference  Disparate Impact
Sex vs. Prediction                        1.178995          1.287554
Race vs. Prediction                       0.545521          1.126932
                     Statistical Parity Difference  Disparate Impact
Sex vs. Prediction                        0.000119          1.000118
Race vs. Prediction                       0.000311          1.000311


## Table Creation

In [14]:
def create_summary_report(protected_class_name, metric_name, original_val, transformed_val, original_pred_val, transformed_pred_val):
    """Creates a summary DataFrame for a given metric, matching the FAQ format."""
    stages = [
        'Original Dataset', 'After Transforming Dataset', 
        'After Training Classifier on Original Dataset', 
        'After Training Classifier on Transformed Dataset'
    ]
    values = [original_val, transformed_val, original_pred_val, transformed_pred_val]
    
    report_df = pd.DataFrame({'Stage': stages, metric_name: values})
    report_df['Change compared to previous'] = report_df[metric_name].diff()
    
    print(f"\n\n--- Detailed Summary for: {protected_class_name} - {metric_name} ---")
    print(report_df.to_string(index=False))

In [15]:
# Sex
sex_spd_orig = original_metrics['Sex vs. Action Taken']['Statistical Parity Difference']
sex_di_orig = original_metrics['Sex vs. Action Taken']['Disparate Impact']
sex_spd_trans = transformed_metrics['Sex vs. Action Taken']['Statistical Parity Difference']
sex_di_trans = transformed_metrics['Sex vs. Action Taken']['Disparate Impact']
sex_spd_pred_orig = original_pred_metrics['Sex vs. Prediction']['Statistical Parity Difference']
sex_di_pred_orig = original_pred_metrics['Sex vs. Prediction']['Disparate Impact']
sex_spd_pred_trans = transformed_pred_metrics['Sex vs. Prediction']['Statistical Parity Difference']
sex_di_pred_trans = transformed_pred_metrics['Sex vs. Prediction']['Disparate Impact']

# Race
race_spd_orig = original_metrics['Race vs. Action Taken']['Statistical Parity Difference']
race_di_orig = original_metrics['Race vs. Action Taken']['Disparate Impact']
race_spd_trans = transformed_metrics['Race vs. Action Taken']['Statistical Parity Difference']
race_di_trans = transformed_metrics['Race vs. Action Taken']['Disparate Impact']
race_spd_pred_orig = original_pred_metrics['Race vs. Prediction']['Statistical Parity Difference']
race_di_pred_orig = original_pred_metrics['Race vs. Prediction']['Disparate Impact']
race_spd_pred_trans = transformed_pred_metrics['Race vs. Prediction']['Statistical Parity Difference']
race_di_pred_trans = transformed_pred_metrics['Race vs. Prediction']['Disparate Impact']


In [16]:
# --- Generate the four required summary tables ---
create_summary_report("Sex", "Statistical Parity Difference", sex_spd_orig, sex_spd_trans, sex_spd_pred_orig, sex_spd_pred_trans)
create_summary_report("Sex", "Disparate Impact", sex_di_orig, sex_di_trans, sex_di_pred_orig, sex_di_pred_trans)
create_summary_report("Race", "Statistical Parity Difference", race_spd_orig, race_spd_trans, race_spd_pred_orig, race_spd_pred_trans)
create_summary_report("Race", "Disparate Impact", race_di_orig, race_di_trans, race_di_pred_orig, race_di_pred_trans)





--- Detailed Summary for: Sex - Statistical Parity Difference ---
                                           Stage  Statistical Parity Difference  Change compared to previous
                                Original Dataset                       0.082922                          NaN
                      After Transforming Dataset                       0.048991                    -0.033931
   After Training Classifier on Original Dataset                       1.178995                     1.130004
After Training Classifier on Transformed Dataset                       0.000119                    -1.178877


--- Detailed Summary for: Sex - Disparate Impact ---
                                           Stage  Disparate Impact  Change compared to previous
                                Original Dataset          1.042136                          NaN
                      After Transforming Dataset          1.024750                    -0.017386
   After Training Classifier on Original Dat